In [1]:
%load_ext autoreload
%autoreload 2
%cd ~/dev/neurad-studio

/home/s0001899/dev/neurad-studio


In [2]:
import torch
from torch import nn
from pathlib import Path
from nerfstudio.models.diffusion import SDPipe, read_yaml, read_image
import matplotlib.pyplot as plt
import torchvision.transforms.v2 as tvtf

from torchmetrics.image.psnr import PeakSignalNoiseRatio
from torchmetrics.image.ssim import StructuralSimilarityIndexMeasure


cp_path = Path("/home/s0001899/dev/output/pytorch_lora_weights.safetensors")
diffusion_config_path = Path("configs/diffusion_model_configs.yml")

diff_config = read_yaml(diffusion_config_path)

In [3]:
pipe = SDPipe({
    "model_name": "sd_base",
    "model_id": "stabilityai/stable-diffusion-2-1",
    "low_mem_mode": True,
    "compile_model": False,
    "lora_weights": cp_path,
    #"lora_weights": None,
})

/home/s0001899/dev/neurad-studio/neurad-env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [23]:
imgs_path = Path("/home/s0001899/dev/renders_left_front/rgb")

img_transform = tvtf.Compose((tvtf.ConvertImageDtype(torch.float32), tvtf.CenterCrop((1024, 1024)), tvtf.Resize((512, 512))))


imgs = []

for img in imgs_path.glob("*.jpg"):
    img = read_image(img, img_transform)
    imgs.append(img)

imgs = torch.stack(imgs)    
   
    
samples = {"rgb": imgs}

/home/s0001899/dev/neurad-studio/neurad-env/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [24]:
imgs.size()

torch.Size([40, 3, 512, 512])

In [10]:
outputs = pipe.get_diffusion_output(samples)

AttributeError: 'list' object has no attribute 'shape'